In [ ]:
import geopandas as gpd
import pandas as pd
import glob

In [ ]:
water2014 = gpd.read_file("water2014_clip.shp")

water2014_buffer = gpd.GeoDataFrame(water2014.buffer(500))
water2014_buffer["NAME"] = water2014["NAME"]
water2014_buffer = water2014_buffer.set_geometry(water2014_buffer[0])
water2014_buffer = water2014_buffer[["NAME", "geometry"]]

In [ ]:
water2014_buffer

In [ ]:
water2014_buffer.plot()

In [ ]:
buffer_size = int(input("Provide a distance for the size of the buffer in meters: "))

In [ ]:
water2014 = gpd.read_file("water2014_clip.shp")
water2016 = gpd.read_file("water2016_clip.shp")
water2018 = gpd.read_file("water2018_clip.shp")
water2020 = gpd.read_file("water2020_clip.shp")

In [ ]:
def buffer_lakes(buffer, water_feat):
    lake_buffer = gpd.GeoDataFrame(water_feat.buffer(buffer))
    lake_buffer["NAME"] = water_feat["NAME"]
    lake_buffer = lake_buffer.set_geometry(lake_buffer[0])
    lake_buffer = lake_buffer[["NAME", "geometry"]]
    return lake_buffer

In [ ]:
buffer2014 = buffer_lakes(buffer_size, water2014)
buffer2016 = buffer_lakes(buffer_size, water2016)
buffer2018 = buffer_lakes(buffer_size, water2018)
buffer2020 = buffer_lakes(buffer_size, water2020)

In [ ]:
buffer2014["NAME"]

In [ ]:
# Get all metro data by finding all files ending in _metro.zip
patterns = (gpd.read_file(r'zip://jul19_metro.zip')).to_crs('EPSG:26915')
directory = r'/home/leex6165/gisproj/'
path = f'{directory}*_metro.zip'

In [ ]:
# Ignore for now. Will be used to loop through and spatial join
# get list of all months in directory
months = []
for file in glob.glob(path):
    months.append(file[-15:-10])

In [ ]:
months

In [ ]:
# Spatial Join documentation: 
# https://geopandas.org/reference/geopandas.sjoin.html
test = gpd.sjoin(water2014_buffer, patterns, op='intersects')

#check_results = test.loc[(test['NAME'] == 'Adams Hill Pond')]
#len(check_results)

In [ ]:
# Get counts of points in each lake buffer.
test_group = test.groupby('NAME', as_index=False)['index_right'].count()
test_group = test_group.rename(columns = {'index_right': 'counts'})

In [ ]:
'''Testing how to join counts for each month and how to create one dataset later. 
Thought: create an empy dataframe with lakes as a row and months as header column names.
Append each count to matching lake name for each monthly dataset. 

Other thought- since dataframes can be created with lists. Could create multiple lists
and turn into a dataframe. 

i.e. data = [[lake1, 14, 10], [lake2, 10, 1]]
pd.DataFrame(data, columns=['lake', 'apr19', 'may19'])
'''
test_dict = test_group.to_dict(orient='records')
yearly_dict = [test_dict]
yearly_dict